In [1]:
import pandas as pd

In [8]:
reviews=pd.read_csv('TM_Project\\data\\reviews.csv')

In [5]:
from textblob import TextBlob

In [11]:
reviews['Review'][1472]

"It's an absolute awesome experience.....gone with mom and she loved it.....Alam and around are a very good help vgood v go vgood vvvvvvvvvvvvv"

In [12]:
reviews['Review'][1568]

'gud'

In [13]:
reviews['Review'][4135]

'ntg'

In [14]:
reviews['Review'][2337]


'good good goodgoodgoodgood goodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgood'

In [7]:
def correct_spelling(review):
    return str(TextBlob(review).correct())

In [15]:
for i in [1472, 1568,4135 ,2337]:
    print(correct_spelling(reviews['Review'][i]))

It's an absolute awesome experience.....gone with mon and she loved it.....Slam and around are a very good help good v go good vvvvvvvvvvvvv
god
nag
good good goodgoodgoodgood goodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgood


In [ ]:
pip install autocorrect

In [18]:
from autocorrect import Speller
spell = Speller()
for i in [1472, 1568,4135 ,2337]:
    print(spell(reviews['Review'][i]))

It's an absolute awesome experience.....gone with mom and she loved it.....Alam and around are a very good help good v go good vvvvvvvvvvvvv
gun
ng
good good goodgoodgoodgood goodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgood


In [19]:
import re
import math


def split_in_chunks(text, chunk_size):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i + chunk_size])
    if len(chunks) > 1 and len(chunks[-1]) < 10:
        chunks[-2] += chunks[-1]
        chunks.pop(-1)
    return chunks


def unique_chars_per_chunk_percentage(text, chunk_size):
    chunks = split_in_chunks(text, chunk_size)
    unique_chars_percentages = []
    for chunk in chunks:
        total = len(chunk)
        unique = len(set(chunk))
        unique_chars_percentages.append(unique / total)
    return sum(unique_chars_percentages) / len(unique_chars_percentages) * 100


def vowels_percentage(text):
    vowels = 0
    total = 0
    for c in text:
        if not c.isalpha():
            continue
        total += 1
        if c in "aeiouAEIOU":
            vowels += 1
    if total != 0:
        return vowels / total * 100
    else:
        return 0


def word_to_char_ratio(text):
    chars = len(text)
    words = len([x for x in re.split(r"[\W_]", text) if x.strip() != ""])
    return words / chars * 100


def deviation_score(percentage, lower_bound, upper_bound):
    if percentage < lower_bound:
        return math.log(lower_bound - percentage, lower_bound) * 100
    elif percentage > upper_bound:
        return math.log(percentage - upper_bound, 100 - upper_bound) * 100
    else:
        return 0


def classify(text):
    if text is None or len(text) == 0:
        return 0.0
    ucpcp = unique_chars_per_chunk_percentage(text, 35)
    vp = vowels_percentage(text)
    wtcr = word_to_char_ratio(text)

    ucpcp_dev = max(deviation_score(ucpcp, 45, 50), 1)
    vp_dev = max(deviation_score(vp, 35, 45), 1)
    wtcr_dev = max(deviation_score(wtcr, 15, 20), 1)

    return max((math.log10(ucpcp_dev) + math.log10(vp_dev) +
                math.log10(wtcr_dev)) / 6 * 100, 1)

In [22]:
reviews['Gibberish Score'] = reviews['Review'].apply(lambda x: classify(str(x)))

In [33]:
reviews['Gibberish Score'].describe()

count    10000.000000
mean        16.963934
std         22.688033
min          1.000000
25%          1.000000
50%          1.000000
75%         25.999670
max        100.000000
Name: Gibberish Score, dtype: float64

In [ ]:
import re
import math


def split_in_chunks(text, chunk_size):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i + chunk_size])
    if len(chunks) > 1 and len(chunks[-1]) < 10:
        chunks[-2] += chunks[-1]
        chunks.pop(-1)
    return chunks


def unique_chars_per_chunk_percentage(text, chunk_size):
    chunks = split_in_chunks(text, chunk_size)
    unique_chars_percentages = []
    for chunk in chunks:
        total = len(chunk)
        unique = len(set(chunk))
        unique_chars_percentages.append(unique / total)
    return sum(unique_chars_percentages) / len(unique_chars_percentages) * 100


def vowels_percentage(text):
    vowels = 0
    total = 0
    for c in text:
        if not c.isalpha():
            continue
        total += 1
        if c in "aeiouAEIOU":
            vowels += 1
    if total != 0:
        return vowels / total * 100
    else:
        return 0


def word_to_char_ratio(text):
    chars = len(text)
    words = len([x for x in re.split(r"[\W_]", text) if x.strip() != ""])
    return words / chars * 100


def deviation_score(percentage, lower_bound, upper_bound):
    if percentage < lower_bound:
        return math.log(lower_bound - percentage, lower_bound) * 100
    elif percentage > upper_bound:
        return math.log(percentage - upper_bound, 100 - upper_bound) * 100
    else:
        return 0


def classify(text):
    if text is None or len(text) == 0:
        return 0.0
    ucpcp = unique_chars_per_chunk_percentage(text, 35)
    vp = vowels_percentage(text)
    wtcr = word_to_char_ratio(text)

    ucpcp_dev = max(deviation_score(ucpcp, 45, 50), 1)
    vp_dev = max(deviation_score(vp, 35, 45), 1)
    wtcr_dev = max(deviation_score(wtcr, 15, 20), 1)

    return max((math.log10(ucpcp_dev) + math.log10(vp_dev) +
                math.log10(wtcr_dev)) / 6 * 100, 1)

In [35]:
pd.set_option('display.max_colwidth', None)

In [36]:
print(reviews[reviews['Gibberish Score']>26]['Review'])

0            The ambience was good, food was quite good . had Saturday lunch , which was cost effective .\r\nGood place for a sate brunch. One can also chill with friends and or parents.\r\nWaiter Soumen Das was really courteous and helpful.
2                                                   A must try.. great food great ambience. Thnx for the service by Pradeep and Subroto. My personal recommendation is Penne Alfredo Pasta:) ....... Also the music in the background is amazing.
10                                                                            The service was great and the food was awesome. The service staff Manab and Papiya were very courteous and attentive. I would like to come frequently to this place
25                                                                                    The place is very good.. 5* to the live music.. She is just superb with beautiful voice.. Food is nice n loved the lightning..😍entrance is also very nice..
31                              